<a href="https://colab.research.google.com/github/agitated-deduction/ML_is_mild_lemon/blob/master/edwith/ReLU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

neural network의 학습

인풋 ->네트워크 ->아웃풋
ground-truth - output = loss
d(loss) = gradient
아웃풋 -> 인풋: backpropagation(네트워크 학습)

네트워크가 딥하다면 시그모이드 그래프가 많아지고 그러면 gradient 가 0인게 많아짐. 그럼 vanishing gradient.

Relu
f(x) = max(0, x)
 
tf.keras.activation -> sigmoid, tanh, relu, elu, selu 등 있으니 다양하게 사용해보면서 성능차이 비교

tf.keras.layers -> leaky relu : 0보다 작은 값을 가질때 매우 작은 알파값.



In [0]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist

# Load mnist

In [0]:
def load_mnist():
  (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
  
  #tensorflow가 input으로 받는 데이터는 [batch_size, height, width, channel]이렇게 구성되어 있어야 한다.
  train_data = np.expand_dims(train_data, axis = -1) #[N, 28, 28]  ---> [N, 28, 28, 1] , aixs = -1(==3) 맨 끝에 추가
  test_data = np.expand_dims(test_data, axis = -1)   #gray scale image라 channel 1 생략

  train_data, test_data = normalize(train_data, test_data) #[0~255] ---> [0~1]

  train_labels = to_categorical(train_labels, 10) #[N,]--->[N, 10] 라벨의 개수. One hot encoding
  test_labels = to_categorical(test_labels, 10)

  return train_data, train_labels, test_data, test_labels

In [0]:
def normalize(train_data, test_data):
  train_data = train_data.astype(np.float32)/255.0
  test_data = test_data.astype(np.float32)/255.0

  return train_data, test_data

# Create network

In [0]:
def flatten(): # 쉐입을 펼춰주는 역할
  return tf.keras.layers.Flatten()
def dense(channel, weight_init): #fully connected layer를 사용하기 위해
  return tf.keras.layers.Dense(units = chnnel, use_bias = True, kernel_initializer = weight_init)
  #유닛은 아웃풋으로 나갈 채널 설정, 바이어스 트루이면 사용 펄스면 사용하지 않음, 커널 이니셜라이저 웨이트 이니셜라이저
def relu():
  return tf.keras.layers.Activation(tf.keras.activation.relu)

In [0]:
class create_model(tf.keras.Model): # model을 class타입으로 짤 때에는 tf.keras.Model을 상속해야 한다.
  def __init__(self, label_dim): #label_dim: 네트워크의 logits을 구할때 몇 개의 아웃풋을 낼 것인지.
    super(create_model, self).__init__()

    weight_init = tf.keras.initializers.RandomNormal()
    self.model = tf.keras.Sequential() #리스트 자료구조 타입과 유사
    
    #flatten으로 바꿔주는 이유는 나중에 fully connected layer를 이용하기 위해.
    self.model.add(flatten()) #[N, 28, 28, 1] ->[N, 784]

    for i in range(2):
      #[N, 784] -> [N, 256] -> [N, 256]
      self.model.add(dense(256, weight_init))
      self.model.add(relu())

    self.model.add(dense(label_dim, weight_init)) #[N, 256] -> [N, 10]

  def call(self, x, training = None, mask = None): #모델을 불렀을때 아웃풋을 내는 함수
    x = self.model(x)
    return x

In [0]:
#클래스 말고 함수형으로 짜는 방법, 성능 차이 ㄴㄴ
def create_model(label):
  weight_init = tf.keras.initializers.RandomNormal()

  model = tf.keras.Sequential()
  model.add(flatten)
  
  for i in range(2):
    model.add(dense(256, weight_init))
    model.add(relu())

  model.add(dense(label_dim, weight_init))

  return model

# Define loss

In [0]:
def loss_fn(model, images, labels):
  logits = model(images, training = True) # 모델에 이미지를 넣어서 이미지가 뭔지 logits 값 추출
  #추출한 logits과 label을 softmax_cross_entroy를 이용해 loss를 구함.
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = logints, labels = labels))
  # ex) image :7, label = 0 0 0 0 0 0 0 1 0 0, softmax(logit) = 0.1 0.1 0.0 0.2 0.0 0.0 0.0 0.6 0.0 0.0
  return loss

def accuracy_fn(model, images, labels):
  logits = model(images, trainin = False) 
  prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1)) #argmax 가장 숫자가 큰 값의 위치
  #logits과 labels의 shape 구조 [batch size, label_dim], argmax(, -1) : 값이 가장 큰 label_dim 위치. 
  #prediction은 true false 로 나옴
  accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32)) #typecast로 boolean을 숫자로 변환
  return accuracy

def grad(model, images, labels):
  with tf.GradientTape() as tape:
    loss = loss_fn(model, images, labels)
  return tape.gradient(loss, model.variables)


# Experiments(parameters)

In [0]:
#dataset
train_x, train_y, test_x, test_y = load_mnist()  #6만 , 1만
#parameters
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) //batch_size

label_dim = 10

#Graph input using dataset API
#6만개의 트레이닝 이미지를 넣기에는 메모리에 부담, batch사이즈 만큼 줌.
#shuffle : #데이터셋 섞음. 값음 inputdata보다 숫자가 크면 됨.
#prefetch : 메모리에 미리 배치사이즈만큼 올려두라는 명령어. 학습 속도를 빠르게 함.
#batch().repeat : #배치를 몇개만큼 반복.
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

# Experiments(model)

In [22]:
#dataset iterator
#train_iterator = train_dataset.make_one_shot_iterator()
#test_iterator = test_dataset.make_one_shot_iterator()

#model
network = create_model(label_dim)

#training 
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_softmax'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

TypeError: ignored

# Experiments(Eager mode)

In [0]:
#네트워크가 학습을 하다가 중간에 끊겼을 때 다시 재학습을 원하면 변경이 되었던 weight들을 불러내는 역할
#즉 학습되어서 저장된 weight들을 다시 불러내는데 도움을 주는 역할
checkpoint = tf.train.Checkpoint(dnn=network)
#각각의 weight의 iteration을 알려줌
global_step = tf.train.create_global_step()


for epoch in range(start_epoch, training_epochs):
  for idx in range(start_iteration, training_iterations):
    train_input, train_label = train_iterator.get_next()

    grads = grad(network, train_input, train_label)
    #optimizer에 gradient적용 후 네트워크 학습을 시킨다.
    optimizer.apply_gradients(grads_and_vars = zip(grads, network.variables), global_step = global_step)

    train_loss = loss_fn(network, train_input, train_label)
    train_accuracy = accuracy_fn(network, train_input, train_label)

    test_input, test_label = test.iterator.get_next()
    test_accuracy = accuracy_fn(network, test_input, test_label)
    print("Epoch: {} {}/{}, train loss: {}, train_accuracy: {}, test accuracy: {}".format(epoch, idx, training_iterations, train_loss, train_accuracy, test_accuracy))
    counter+=1

checkpoint.save(file_prefix = checkpoint_prefix + '-{}'.format(counter))

일단 따라서 짰는데 안됨 나중에 다시 수정하기